## Visual Transformer for Skin Cancer Detection

In [25]:
import evaluate
import torch
from transformers import ViTForImageClassification, Trainer, TrainingArguments
from transformers import ViTImageProcessor
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
import datasets
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import h5py
import cv2
from PIL import Image
from transformers import pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import safetensors.torch
from sklearn.metrics import accuracy_score
import random

In [2]:
#loading data
BASE_PATH = "isic-2024-challenge"

# Train + Valid
df = pd.read_csv(f'{BASE_PATH}/train-metadata.csv')
df = df.ffill()
display(df.head(2))

# Testing
testing_df = pd.read_csv(f'{BASE_PATH}/test-metadata.csv')
testing_df = testing_df.ffill()
display(testing_df.head(2))

/scratch/300484.1.ood/ipykernel_1502367/4180780485.py:5: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{BASE_PATH}/train-metadata.csv')


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455


,isic_id,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015657,IP_6074337,45.0,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,20.007270,...,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015729,IP_1664139,35.0,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,9.657964,...,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY


In [3]:
#Handle Class Imbalance
print("Class Distribution Before Sampling (%):")
display(df.target.value_counts(normalize=True)*100)
seed = 1
neg_sample = .01
pos_sample = 5.0
# Sampling
positive_df = df.query("target==0").sample(frac=neg_sample, random_state=seed)
negative_df = df.query("target==1").sample(frac=pos_sample, replace=True, random_state=seed)
df = pd.concat([positive_df, negative_df], axis=0).sample(frac=1.0)

print("\nClass Distribution After Sampling (%):")
display(df.target.value_counts(normalize=True)*100)

# Assume df is your DataFrame and 'target' is the column with class labels
class_weights = compute_class_weight('balanced', classes=np.unique(df['target']), y=df['target'])
class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)

Class Distribution Before Sampling (%):


target
0    99.902009
1     0.097991
Name: proportion, dtype: float64


Class Distribution After Sampling (%):


target
0    67.09645
1    32.90355
Name: proportion, dtype: float64

Class Weights: {0: 0.7451959071624656, 1: 1.519592875318066}


In [27]:
from sklearn.model_selection import StratifiedGroupKFold

def get_k_data(df, k):
    training_validation_hdf5 = h5py.File(f"{BASE_PATH}/train-image.hdf5", 'r')

    # Reset index to ensure a continuous index
    df = df.reset_index(drop=True)
    df["fold"] = -1

    # Set up the StratifiedGroupKFold with 5 splits
    sgkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=seed)

    # Assign fold numbers to each data point
    for i, (training_idx, validation_idx) in enumerate(sgkf.split(df, y=df.target, groups=df.patient_id)):
        df.loc[validation_idx, "fold"] = int(i)
    # Define the train, validation, and test sets
    # Use fold 0 for test, fold 1 for validation, and remaining folds for training
    #change these for each trial, testing all test_fold = [1,2,3,4,5]
    test_fold = k
    valid_fold = random.choice([num for num in range(5) if num != k])

    # Create the training, validation, and test sets
    training_df = df.query(f"fold != {valid_fold} and fold != {test_fold}")
    validation_df = df.query(f"fold == {valid_fold}")
    test_df = df.query(f"fold == {test_fold}")

    # Print the number of samples in each set
    print(f"# Num Train: {len(training_df)} | Num Valid: {len(validation_df)} | Num Test: {len(test_df)}")
    return training_df, validation_df, test_df


In [22]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

transform = Compose([
        Resize((224, 224)),
        ToTensor(),
        Normalize(mean=processor.image_mean, std=processor.image_std),
    ])

def preprocess_images(example):
        byte_string = training_validation_hdf5[example["isic_id"]][()]
        nparr = np.frombuffer(byte_string, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)[...,::-1]
        image = Image.fromarray(image)
        example['pixel_values'] = transform(image)
        return example

def get_datasets(df, k):
        training_df,validation_df,test_df = get_k_data(df, k)

        ds_training = datasets.Dataset.from_pandas(pd.DataFrame(data=training_df))
        ds_valid = datasets.Dataset.from_pandas(pd.DataFrame(data=validation_df))
        ds_test = datasets.Dataset.from_pandas(pd.DataFrame(data=test_df))

        ds_training = ds_training.map(preprocess_images, batched=False)
        ds_valid = ds_valid.map(preprocess_images, batched=False)
        ds_test = ds_test.map(preprocess_images, batched=False)

        return ds_training, ds_valid, ds_test

In [29]:
#accuracy metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
     

In [15]:
def process_test_set(ds, num_samples):
    inputs = []
    labels = []
    for i in range(num_samples):
        byte_string = training_validation_hdf5[ds[i]["isic_id"]][()]
        nparr = np.frombuffer(byte_string, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)[...,::-1]
        image = Image.fromarray(image)
        inputs.append(image)
        labels.append(ds[i]["target"]) 
    return inputs, labels

In [23]:
def test_validate(model):
    inputs, y_true = process_test_set(ds_test, 1000)

    image_classifier = pipeline("image-classification", model ,image_processor=processor)
    predictions = image_classifier(inputs)
    #select highest labels
    predictions = [max(item, key=lambda x: x['score'])['label'] for item in predictions]
    #convert from LABEL_0, LABEL_1 to 0,1
    predictions = [1 if item == 'LABEL_1' else 0 for item in predictions]  
    accuracy = accuracy_score(y_true, predictions)
    #plot confusion matrix
    confusion_metric = evaluate.load("confusion_matrix")
    confusion_matrix = confusion_metric.compute(predictions=predictions, references=y_true)
    matrix = confusion_matrix['confusion_matrix']
    if 'labels' in confusion_matrix:
        labels = confusion_matrix['labels']
    else:
        labels = np.unique(predictions + y_true)
    sns.heatmap(matrix, annot=True, fmt="d", xticklabels=labels, yticklabels=labels, cmap="Blues")
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.title("Confusion Matrix")
    plt.show()  
    return accuracy

In [30]:
model_name='google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name)

# Make sure its the right tensor types
def collate_fn(batch):
    pixel_values = torch.stack([torch.tensor(example['pixel_values']) for example in batch])
    labels = torch.tensor([example['target'] for example in batch])
    return {'pixel_values': pixel_values, 'labels': labels}


for k in range(1):
    ds_training, ds_valid, ds_test = get_datasets(df, k)
    
    # Define training arguments
    training_args = TrainingArguments(
        output_dir='./vit-finetuned-agent0',
        metric_for_best_model = "accuracy",
        per_device_train_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=20,
        logging_dir='./logs',
        logging_steps=10,
        load_best_model_at_end=True,
        remove_unused_columns=False,
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds_training,
        eval_dataset=ds_valid,
        data_collator=collate_fn,
        tokenizer=processor,
        compute_metrics=compute_metrics
    )

    #train da model
    trainer.train()

    # Save da fine-tuned model
    trainer.save_model('./vit-finetuned-agentO')

    trainer.eval_dataset=ds_test
    trainer.evaluate()

    test_validate(model)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Num Train: 3426 | Num Valid: 1287 | Num Test: 1259


Map:   0%|          | 0/3426 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Map:   0%|          | 0/1259 [00:00<?, ? examples/s]

/projectnb/cs640grp/students/samwu/AgentO/.venv/lib64/python3.11/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/scratch/300484.1.ood/ipykernel_1502367/742869072.py:30: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
